In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df=pd.read_csv("D:/Semester5Project/og.csv",low_memory=False)

In [3]:
df.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,NaN,0


In [4]:
df.isnull().any()

srcip               False
sport               False
dstip               False
dsport              False
proto               False
state               False
dur                 False
sbytes              False
dbytes              False
sttl                False
dttl                False
sloss               False
dloss               False
service             False
Sload               False
Dload               False
Spkts               False
Dpkts               False
swin                False
dwin                False
stcpb               False
dtcpb               False
smeansz             False
dmeansz             False
trans_depth         False
res_bdy_len         False
Sjit                False
Djit                False
Stime               False
Ltime               False
Sintpkt             False
Dintpkt             False
tcprtt              False
synack              False
ackdat              False
is_sm_ips_ports     False
ct_state_ttl        False
ct_flw_http_mthd     True
is_ftp_login

In [12]:
df.drop(index=df[df.duplicated()].index,inplace=True)

In [ ]:
df.shape

Dropping columns with too many NaN values and attack_cat 

In [15]:
df.drop(columns=['ct_flw_http_mthd','is_ftp_login','attack_cat','ct_ftp_cmd'],inplace=True)

KeyError: "['ct_flw_http_mthd' 'is_ftp_login' 'attack_cat' 'ct_ftp_cmd'] not found in axis"

Dropping the IP Addresses as they are not useful

In [6]:
df.drop(columns=['srcip','dstip'],inplace=True)

In [22]:
df['service']=df['service'].apply(lambda x: 'unknown' if x=='-' else x)

In [36]:
df['sport']=df['sport'].apply(lambda x: '12' if x=='0x000c' else x)
df['sport']=df['sport'].apply(lambda x: '11' if x=='0x000b' else x)
df['sport']=df['sport'].apply(lambda x: None if x=='-' else x)
df['sport']=df['sport'].astype(int)

In [67]:
df['dsport']=df['dsport'].apply(lambda x: '52233' if x=='0xcc09' else x)
df['dsport']=df['dsport'].apply(lambda x: '49320' if x=='0xc0a8' else x)
df['dsport']=df['dsport'].apply(lambda x: None if x=='0x20205321' else x)
df['dsport']=df['dsport'].apply(lambda x: None if x=='-' else x)
df['dsport']=df['dsport'].astype(int)

ValueError: invalid literal for int() with base 10: '-'

In [44]:
df.select_dtypes(exclude=np.number)

,dsport,proto,state,service
0,1024,udp,CON,unknown
1,53,udp,CON,dns
2,53,udp,CON,dns
3,53,udp,CON,dns
4,111,udp,CON,unknown
...,...,...,...,...
2540038,43433,tcp,FIN,ftp-data
2540039,21,tcp,CON,ftp
2540040,21,tcp,CON,ftp
2540041,80,tcp,CON,http
